# GPT2でファインチューニングしたモデルから文章生成する。

公開されているRinnaの学習モデルをファインチューニングしたモデルから文章の生成を行う。

## 環境

 - Clab PRO
 - Google Drive

## ディレクトリ

 - ./work : transformersのcode（一部改変して使うのでGoogle Driveで永続化)
 - ./output : 学習したモデルを保存

## 学習ファイルの形式

人物：セリフ

例「アムロ：シャア、なぜわからないんだ」

## 参考にしたサイト

 - https://note.com/npaka/n/n8a435f0c8f69


In [ ]:
# データの永続化(Google Driveへの接続)
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
%cd '/content/drive/My Drive/work/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/work


In [ ]:
# Huggingface Transformers
# e オプションができなくなってた・・・。
!git clone https://github.com/huggingface/transformers -b v4.4.2
!pip install transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [ ]:
# 作業フォルダへ
%cd '/content/drive/My Drive/work/'

/content/drive/My Drive/work


In [ ]:
# Huggingface Datasets
!pip install datasets==1.2.1
# Sentencepiece
!pip install sentencepiece==0.1.91

# Tokenizerの変更

「rinna」の日本語GPT-2モデルは「AutoTokenizer」ではなく「T5Tokenizer」を使っている。
ファインチューニングに用いる「./transformers/examples/language-modeling/run_clm.py」を予め編集しておく。

トレーニング用セリフ集は「train_g01.txt」とした。

In [ ]:
%%time

# ファインチューニングの実行
# ./output　があっても上書きする　--overwrite_output
!python ./transformers/examples/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-small \
    --train_file=train_g01.txt \
    --validation_file=train_g01.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=30 \
    --save_steps=10000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=output/ \
    --overwrite_output_dir \
    --use_fast_tokenizer=False

05/04/2022 01:03:39 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
05/04/2022 01:03:39 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=F

### Error で動かないので対策。

In [ ]:
!pip uninstall -y numpy
!pip install -U numpy

Found existing installation: numpy 1.21.6
Uninstalling numpy-1.21.6:
  Successfully uninstalled numpy-1.21.6
     |████████████████████████████████| 15.7 MB 345 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


# テキスト生成のテスト

In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-small")
model = AutoModelForCausalLM.from_pretrained("output/")

# できたモデルから生成
input = tokenizer.encode(r"シャア:ララァは母になるべき女だったんだ", return_tensors="pt")
output = model.generate(input,
                        do_sample=True,
                        top_p=0.90,
                        top_k=0,
                        max_length=100,
                        skip_special_tokens=True,
                        pad_token_id=tokenizer.pad_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bad_word_ids=[[tokenizer.unk_token_id]]
                        )

# セリフ用の整形はせずにとりあえず、出力。
print(tokenizer.decode(output[0], skip_special_tokens=True))

シャア:ララァは母になるべき女だったんだ アムロ そうか
